<a href="https://colab.research.google.com/github/anika0520/Infosys_springboard-6.0-assignments/blob/main/multilingual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install scikit-learn googletrans==4.0.0-rc1 indictrans2-ai4bharat langdetect


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from indictrans2 import IndicTranslator
from langdetect import detect
import numpy as np

In [4]:
diseases = {
    "Tomato Blight": {
        "symptom": {
            "en": "brown spots on the leaves",
            "hi": "पत्तियों पर भूरे धब्बे",
            "bn": "পাতায় বাদামি দাগ",
            "te": "ఆకులపై గోధుమ రంగు మచ్చలు"
        },
        "prevention": {
            "en": "Use resistant varieties, remove infected plants, and apply fungicide.",
            "hi": "प्रतिरोधी किस्मों का उपयोग करें, संक्रमित पौधों को हटाएं और फफूंदनाशी का उपयोग करें।",
            "bn": "প্রতিরোধী জাত ব্যবহার করুন, আক্রান্ত গাছ অপসারণ করুন এবং ছত্রাকনাশক প্রয়োগ করুন।",
            "te": "ప్రతిఘటించే రకాలను ఉపయోగించండి, సంక్రామిత మొక్కలను తీసివేయండి మరియు ఫంగిసైడ్ వాడండి."
        }
    },
    "Powdery Mildew": {
        "symptom": {
            "en": "white powdery spots on the leaves",
            "hi": "पत्तियों पर सफेद चूर्ण जैसे धब्बे",
            "bn": "পাতায় সাদা গুঁড়ো দাগ",
            "te": "ఆకులపై తెల్లటి పొడి మచ్చలు"
        },
        "prevention": {
            "en": "Ensure proper spacing, avoid overhead watering, and use fungicide.",
            "hi": "उचित दूरी सुनिश्चित करें, ऊपर से पानी देने से बचें और फफूंदनाशी का उपयोग करें।",
            "bn": "সঠিক দূরত্ব নিশ্চিত করুন, উপরে থেকে জল দেওয়া এড়িয়ে চলুন এবং ছত্রাকনাশক ব্যবহার করুন।",
            "te": "సరైన దూరం ఉంచండి, పై నుండి నీరు పోయడం నివారించండి మరియు ఫంగిసైడ్ వాడండి."
        }
    },
    "Leaf Spot": {
        "symptom": {
            "en": "dark spots on the leaves",
            "hi": "पत्तियों पर काले धब्बे",
            "bn": "পাতায় কালো দাগ",
            "te": "ఆకులపై నల్ల మచ్చలు"
        },
        "prevention": {
            "en": "Remove infected leaves, rotate crops, and use copper-based sprays.",
            "hi": "संक्रमित पत्ते हटाएं, फसल चक्र अपनाएं और तांबे पर आधारित स्प्रे का उपयोग करें।",
            "bn": "আক্রান্ত পাতা সরান, ফসল আবর্তন করুন এবং তামা-ভিত্তিক স্প্রে ব্যবহার করুন।",
            "te": "సంక్రామిత ఆకులను తీసివేయండి, పంటల మార్పిడి చేయండి మరియు రాగి స్ప్రే వాడండి."
        }
    },
    "Rust": {
        "symptom": {
            "en": "orange pustules on the leaves",
            "hi": "पत्तियों पर नारंगी फफोले",
            "bn": "পাতায় কমলা ফোসকা",
            "te": "ఆకులపై నారింజ బుడగలు"
        },
        "prevention": {
            "en": "Use resistant varieties, remove infected leaves, and apply fungicide.",
            "hi": "प्रतिरोधी किस्मों का उपयोग करें, संक्रमित पत्तों को हटाएं और फफूंदनाशी का प्रयोग करें।",
            "bn": "প্রতিরোধী জাত ব্যবহার করুন, আক্রান্ত পাতা সরান এবং ছত্রাকনাশক প্রয়োগ করুন।",
            "te": "ప్రతిఘటించే రకాలను ఉపయోగించండి, సంక్రామిత ఆకులను తొలగించండి మరియు ఫంగిసైడ్ వాడండి."
        }
    },
    "Downy Mildew": {
        "symptom": {
            "en": "yellow angular spots on the leaves",
            "hi": "पत्तियों पर पीले कोणीय धब्बे",
            "bn": "পাতায় হলুদ কোণার দাগ",
            "te": "ఆకులపై పసుపు మచ్చలు"
        },
        "prevention": {
            "en": "Improve air circulation, avoid overhead watering, and use fungicide.",
            "hi": "हवा का संचार बढ़ाएं, ऊपर से पानी देने से बचें और फफूंदनाशी का उपयोग करें।",
            "bn": "বায়ু চলাচল বাড়ান, উপরে থেকে জল দেওয়া এড়িয়ে চলুন এবং ছত্রাকনাশক ব্যবহার করুন।",
            "te": "గాలి ప్రవాహం మెరుగుపరచండి, పై నుండి నీరు పోయడం నివారించండి మరియు ఫంగిసైడ్ వాడండి."
        }
    },
    "Fusarium Wilt": {
        "symptom": {
            "en": "yellowing and wilting of the leaves",
            "hi": "पत्तियों का पीला पड़ना और मुरझाना",
            "bn": "পাতা হলুদ হয়ে যাওয়া এবং মুরঝিয়ে যাওয়া",
            "te": "ఆకులు పసుపు రంగులోకి మారడం మరియు వాడిపోవడం"
        },
        "prevention": {
            "en": "Plant resistant varieties and practice crop rotation.",
            "hi": "प्रतिरोधी किस्मों का उपयोग करें और फसल चक्र अपनाएं।",
            "bn": "প্রতিরোধী জাত ব্যবহার করুন এবং ফসল আবর্তন করুন।",
            "te": "ప్రతిఘటించే రకాలను ఉపయోగించండి మరియు పంటల మార్పిడి చేయండి."
        }
    },
    "Bacterial Spot": {
        "symptom": {
            "en": "water-soaked lesions on the leaves",
            "hi": "पत्तियों पर पानी से भीगे हुए धब्बे",
            "bn": "পাতায় পানিতে ভেজা দাগ",
            "te": "ఆకులపై నీటితో నిండి ఉన్న మచ్చలు"
        },
        "prevention": {
            "en": "Remove infected leaves, use certified disease-free seeds, and rotate crops.",
            "hi": "संक्रमित पत्ते हटाएं, प्रमाणित रोगमुक्त बीज का उपयोग करें और फसल चक्र अपनाएं।",
            "bn": "আক্রান্ত পাতা সরান, শংসাপত্রপ্রাপ্ত রোগমুক্ত বীজ ব্যবহার করুন এবং ফসল আবর্তন করুন।",
            "te": "సంక్రామిత ఆకులను తీసివేయండి, సర్టిఫైడ్ రోగరహిత బీజాలు ఉపయోగించండి మరియు పంటల మార్పిడి చేయండి."
        }
    },
    "Leaf Curl": {
        "symptom": {
            "en": "curling of the leaves",
            "hi": "पत्तियों का मुड़ना",
            "bn": "পাতা কুঁকড়ে যাওয়া",
            "te": "ఆకులు ముడుచుకోవడం"
        },
        "prevention": {
            "en": "Avoid overhead irrigation and use resistant varieties.",
            "hi": "ऊपर से सिंचाई से बचें और प्रतिरोधी किस्मों का उपयोग करें।",
            "bn": "উপর থেকে সেচ দেওয়া এড়িয়ে চলুন এবং প্রতিরোধী জাত ব্যবহার করুন।",
            "te": "పై నుండి నీరు పోయడం నివారించండి మరియు ప్రతిఘటించే రకాలను ఉపయోగించండి."
        }
    },
    "Blossom End Rot": {
        "symptom": {
            "en": "black, sunken spots on the blossom end of fruit",
            "hi": "फलों के निचले सिरे पर काले और धंसे हुए धब्बे",
            "bn": "ফলের নিচের অংশে কালো, ধসে পড়া দাগ",
            "te": "పండ్ల చివరలో నల్లని, లోపలికి పోయిన మచ్చలు"
        },
        "prevention": {
            "en": "Ensure proper watering and calcium supplementation.",
            "hi": "सही पानी की व्यवस्था और कैल्शियम का उपयोग सुनिश्चित करें।",
            "bn": "সঠিক সেচ এবং ক্যালসিয়াম প্রদান নিশ্চিত করুন।",
            "te": "సరైన నీటిపారుదల మరియు కాల్షియం ఇచ్చే సమర్ధన చేయండి."
        }
    },
    "Verticillium Wilt": {
        "symptom": {
            "en": "stunted growth and yellowing of leaves",
            "hi": "विकास रुक जाना और पत्तों का पीला पड़ना",
            "bn": "বৃদ্ধি বন্ধ হয়ে যাওয়া এবং পাতার হলুদ হওয়া",
            "te": "పెరుగుదల మందగించడం మరియు ఆకులు పసుపు రంగులోకి మారడం"
        },
        "prevention": {
            "en": "Use resistant varieties and practice crop rotation.",
            "hi": "प्रतिरोधी किस्मों का उपयोग करें और फसल चक्र अपनाएं।",
            "bn": "প্রতিরোধী জাত ব্যবহার করুন এবং ফসল আবর্তন করুন।",
            "te": "ప్రతిఘటించే రకాలను ఉపయోగించండి మరియు పంటల మార్పిడి చేయండి."
        }
    }
}

In [5]:
translator = IndicTranslator()

In [6]:
def get_lang(text):
    try:
        code = detect(text)
        if code.startswith('hi'):
            return 'hi'
        elif code.startswith('bn'):
            return 'bn'
        elif code.startswith('te'):
            return 'te'
        elif code.startswith('en'):
            return 'en'
        else:
            return 'en'
    except:
        return 'en'

In [7]:
all_symptoms = []
symptom_map = {}
symptom_lang = []

for d, info in diseases.items():
    for lang, s_text in info['symptom'].items():
        all_symptoms.append(s_text)
        symptom_map[s_text] = d
        symptom_lang.append(lang)

In [8]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(all_symptoms)

In [9]:
def translate_text(text, src, tgt):
    if src == tgt:
        return text
    try:
        result = translator.translate(text, source_lang=src, target_lang=tgt)
        if isinstance(result, list):
            return result[0]
        return result
    except:
        return text

In [58]:
def find_disease(user_input, out_lang):
    user_lang = get_lang(user_input)
    user_input_en = translate_text(user_input, user_lang, 'en')

    global all_symptoms_en, vec_en, X_en
    if 'X_en' not in globals():
        all_symptoms_en = [translate_text(s, l, 'en') for s, l in zip(all_symptoms, symptom_lang)]
        vec_en = TfidfVectorizer()
        X_en = vec_en.fit_transform(all_symptoms_en)

    input_vec = vec_en.transform([user_input_en])
    scores = cosine_similarity(input_vec, X_en)[0]

    idx = np.argmax(scores)
    conf = scores[idx]

    matched_en = all_symptoms_en[idx]
    matched_orig = all_symptoms[idx]
    disease_name = symptom_map[matched_orig]

    disease_out = translate_text(disease_name, 'en', out_lang)

    prevent_texts = diseases[disease_name]['prevention']
    prevent = prevent_texts.get(out_lang, prevent_texts.get('en', 'No info '))
    if out_lang != 'en' and out_lang not in prevent_texts:
        prevent = translate_text(prevent, 'en', out_lang)

    return disease_out, prevent, conf

In [59]:
lang_msg = {
    'en': "Choose language: 1. English, 2. Hindi, 3. Bengali, 4. Telugu: ",
    'hi': "भाषा चुनें: 1. English, 2. Hindi, 3. Bengali, 4. Telugu: ",
    'bn': "ভাষা নির্বাচন করুন: 1. English, 2. Hindi, 3. Bengali, 4. Telugu: ",
    'te': "భాషను ఎంచుకోండి: 1. English, 2. Hindi, 3. Bengali, 4. Telugu: "
}

In [60]:
labels = {
    'en': {'disease': "Disease:", 'prevention': "Prevention:", 'conf': "Confidence:"},
    'hi': {'disease': "रोग:", 'prevention': "रोकथाम:", 'conf': "विश्वास:"},
    'bn': {'disease': "রোগ:", 'prevention': "প্রতিরোধ:", 'conf': "আত্মবিশ্বাস:"},
    'te': {'disease': "రోగం:", 'prevention': "నివారణ:", 'conf': "నమ్మకం:"}
}

In [61]:
print("\nFarmer Bot ready! Type 'exit' to quit.\n")



Farmer Bot ready! Type 'exit' to quit.



In [75]:
chosen_lang = 'en'
lang_choice = input(lang_msg['en']).strip()


Choose language: 1. English, 2. Hindi, 3. Bengali, 4. Telugu: 2


In [76]:
lang_map = {'1': 'en', '2': 'hi', '3': 'bn', '4': 'te'}
if lang_choice in lang_map:
    chosen_lang = lang_map[lang_choice]
else:
    print("Invalid choice. Defaulting to English.")


In [77]:
symptom_msg = {
    'en': "Enter plant symptom: ",
    'hi': "पौधे का लक्षण दर्ज करें: ",
    'bn': "উদ্ভিদের লক্ষণ লিখুন: ",
    'te': "మొక్క లక్షణాన్ని ఎంటర్ చేయండి: "
}

In [78]:
while True:
    text = input(symptom_msg[chosen_lang]).strip()
    if text.lower() == 'exit':
        break

    disease, prevent, conf = find_disease(text, chosen_lang)
    lbl = labels[chosen_lang]

    print(f"\n{lbl['disease']} {disease}")
    print(f"{lbl['prevention']} {prevent}")

पौधे का लक्षण दर्ज करें: पत्तों में जीवाणुओं के धब्बे हैं

रोग: लीफ कर्ल
रोकथाम: ऊपर से सिंचाई से बचें और प्रतिरोधी किस्मों का उपयोग करें।
पौधे का लक्षण दर्ज करें: exit
